In [1]:
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
from operator import itemgetter
import warnings
import pickle
from Bio import Seq
from Bio import SeqIO
import pysam
import itertools
import scipy
from scipy import stats
from scipy.interpolate import interp1d
from scipy.stats import chi2_contingency
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
/lustre/user/lulab/doushq/wuxk/software/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
"""
PARAMETERS FOR DETECT
"""

#specify a path to the fasta file (DNA sequence of protein coding sequences)
path_to_fasta = '../Data/cds_pools_1009.fasta'

#path to MaxQuant's table, allPeptides.txt
path_to_allPeptides = '../Data/allPeptides.txt'

#m/z tolerance, used to filter DP–BP couples that resemble substitutions
#and exclude couples that resemble known PTM
tol = 0.005

#these parameters control the filtering of n-term and c-term modifications
n_term_prob_cutoff = 0.05
c_term_prob_cutoff = 0.05

#defines the minimal threshold on MaxQuant's localisation probability. Only
#DP–BP peptides for which the localisation of the modification has be been
#determined with higher confidence will be retained. 
positional_probability_cutoff = 0.95

#FDR for the final target-decoy FDR procedure
fdr = 0.01

# danger_mods were download from https://github.com/ernestmordret/substitutions/blob/master/danger_mods, 
# could also be built by yourself using the script (https://github.com/ernestmordret/substitutions/blob/master/params.py)
path_to_danger_mods = '../Data/danger_mods'

"""
PARAMETERS FOR QUANTIFY
"""

excluded_samples = []

#path to MaxQuant's table, evidence.txt
path_to_evidence = '../Data/evidence.txt'

#path to MaxQuant's table, matchedFeatures.txt
path_to_matched_features = '../Data/matchedFeatures.txt'

#path to MaxQuant's table, peptides.txt
path_to_peptides = '../Data/peptides.txt'

#m/z tolerance for the fetching unidentified features
mz_tol = 10 * 10**-6 # 10 ppm

#retention time tolerance for the fetching unidentified features
rt_tol = 0.3 # min

#path to MaxQuant's table, proteinGroups.txt
path_to_proteinGroups = '../Data/proteinGroups.txt'

# path to MaxQuant's summary, summary.txt
path_to_summary = '../Data/summary.txt'

### Substitution detection

In [3]:
def codonify(seq):
    """
    input: a nucleotide sequence (not necessarily a string)
    output: a list of codons
    """
    seq = str(seq)
    l = len(seq)
    return [seq[i:i+3] for i in range(0,l,3)]

def find_proteins(base_seq):
    """
    input: a peptide sequence (string)
    output: the names of proteins containing that sequence 
    """
    tbr = " ".join([names_list[i] for i in np.searchsorted(boundaries_aa-1, SA_search(base_seq, W_aa, sa))])
    if tbr.strip(" ") == '':
        return ''
    else:
        return tbr
    
def fetch_codon(base_seq, modified_pos):
    """
    input: the original aa sequence of a peptide (base_seq),
            and the relative position of the modification.
    output: returns the list of all codons possibly associated 
            with the substitution, presented as a string separated
            by white spaces.
    """
    possible_codons = []
    proteins = find_proteins(base_seq)
    if proteins:
        proteins = proteins.split(" ")
    else:
        return '_'
    for p in proteins:
        if p in record_dict:
            s = record_dict[p].seq
            seq_i = s.translate().find(base_seq)
            i = seq_i + modified_pos
            possible_codons.append(codonify(s)[i])
        else:
            possible_codons.append('_')
    return " ".join(possible_codons)

def fetch_best_codons(modified_seq):
    """
    input: a modified sequence, e.g. LQV(0.91)A(0.09)EK
    output: the list of codons associated with the most likely
            position
    """
    possible_sites = re.findall('\(([^\)]+)\)', modified_seq)
    best_site = np.argmax([float(i) for i in possible_sites]) # 返回数组中最大概率的氨基酸的索引
    modified_pos_prime = [m.start()-1 for m in re.finditer('\(',modified_seq) ][best_site]
    modified_pos = len(re.sub('\(([^\)]+)\)', '',modified_seq[:modified_pos_prime]))
    base_seq = re.sub('\(([^\)]+)\)', '', modified_seq)
    return fetch_codon(base_seq, modified_pos)

def find_substitution_position_local(modified_seq, protein):
    """
    returns the position of a substitutions relative to the start
    of the protein sequence
    """
    possible_sites = re.findall('\(([^\)]+)\)', modified_seq)
    best_site = np.argmax([float(i) for i in possible_sites])
    modified_pos_prime = [m.start()-1 for m in re.finditer('\(',modified_seq) ][best_site]
    modified_pos = len(re.sub('\(([^\)]+)\)', '', modified_seq[:modified_pos_prime]))
    base_seq = re.sub('\(([^\)]+)\)', '', modified_seq)
    s = record_dict[protein].seq
    seq_i = s.translate().find(base_seq)
    i = seq_i + modified_pos
    return i

def find_positions_local(modified_seq, proteins):
    """
    returns the position of a substitutions relative to the start
    of the protein sequence, across all the codons
    """
    positions = []
    for prot in proteins.split(" "):
        positions.append(str(find_substitution_position_local(modified_seq, prot)))
    return " ".join(positions)

def is_gene(record):
    """
    return True if the sequence satisfies the CDS requirement
    """
    if len(record.seq)%3 != 0:
        return False
    if not record.seq[:3] in {'ATG','GTG','TTG','ATT','CTG'}:
        return False
    if record.seq[-3:].translate()!='*':
        return False
    return True

def refine_localization_probabilities(modified_seq, threshold = 0.05):
    """
    returns the AAs that were possibly modified (with p > threshold).
    Input: modified sequence (a string of AA with p of each to contain modification: APKV(.7)ML(.3)L means that V was modified with p = .7 and L with p = .3)
    Output: A string with all candidate AAs separated by ';' (V;L).
    """
    modified_sites = [modified_seq[m.start()-1] for m in re.finditer('\(',modified_seq) ]
    weights = [float(i) for i in re.findall('\(([^\)]+)\)',modified_seq)]
    site_probabilities = {}    
    for aa, weight in zip(modified_sites, weights):
        if aa in site_probabilities:
            site_probabilities[aa] += weight
        else:
            site_probabilities[aa] = weight
    return ";".join([k for k,v in site_probabilities.items() if v>threshold])

def c_term_probability(modified_sequence):
    """
    Returns the probability that C term AA was modified.
    """
    if modified_sequence[-1] == ')':
        return float(modified_sequence[:-1].split('(')[-1])
    else:
        return 0.0
    
def n_term_probability(modified_sequence):
    """
    Returns the probability that C term AA was modified.
    """
    if modified_sequence[1] == '(':
        return float(modified_sequence[2:].split(')')[0])
    else:
        return 0.0
    
def is_prot_nterm(sequence):
    """
    Does the peptide originate at the protein's N-term
    """
    for start in SA_search(sequence, W_aa, sa):
        if W_aa[start-1] == '*':
            return True
        if W_aa[start-2] == '*':
            return True
    return False

def is_prot_cterm(sequence):
    """
    """
    l=len(sequence)
    for start in SA_search(sequence, W_aa, sa):
        end = start+l
        if W_aa[end] == '*':
            return True
    return False

def get_codon_table():
    return dict(zip(codons, amino_acids))

def get_inverted_codon_table():
    ct = get_codon_table()
    inv_codon_table = {}
    for k, v in ct.items():
        inv_codon_table[v] = inv_codon_table.get(v, [])
        inv_codon_table[v].append(k)
    return inv_codon_table

# 计算hamming距离
def hamming(s1,s2): return sum(a!=b for a,b in zip(s1,s2))  

def is_mispairing(row):
    """
    Returns whether the substitution is mispairing or misloading, based on the
    near-cognate mask.
    """
    codon = row['codon']
    destination = row['destination']
    if pd.notnull(codon) and pd.notnull(destination):
        if (codon in mask.index) and destination:
            return mask.loc[codon,destination]
        else:
            return 0
    else:
        return float('NaN')
    
def suffix_array(text, _step=32):
    """Analyze all common strings in the text.
    
    Short substrings of the length _step a are first pre-sorted. The are the 
    results repeatedly merged so that the garanteed number of compared
    characters bytes is doubled in every iteration until all substrings are
    sorted exactly.
    
    Arguments:
        text:  The text to be analyzed.
        _step: Is only for optimization and testing. It is the optimal length
               of substrings used for initial pre-sorting. The bigger value is
               faster if there is enough memory. Memory requirements are
               approximately (estimate for 32 bit Python 3.3):
                   len(text) * (29 + (_size + 20 if _size > 2 else 0)) + 1MB
    
    Return value:      (tuple)
      (sa, rsa, lcp)
        sa:  Suffix array                  for i in range(1, size):
               assert text[sa[i-1]:] < text[sa[i]:]
        rsa: Reverse suffix array          for i in range(size):
               assert rsa[sa[i]] == i
        lcp: Longest common prefix         for i in range(1, size):
               assert text[sa[i-1]:sa[i-1]+lcp[i]] == text[sa[i]:sa[i]+lcp[i]]
               if sa[i-1] + lcp[i] < len(text):
                   assert text[sa[i-1] + lcp[i]] < text[sa[i] + lcp[i]]
    >>> suffix_array(text='banana')
    ([5, 3, 1, 0, 4, 2], [3, 2, 5, 1, 4, 0], [0, 1, 3, 0, 0, 2])
    
    Explanation: 'a' < 'ana' < 'anana' < 'banana' < 'na' < 'nana'
    The Longest Common String is 'ana': lcp[2] == 3 == len('ana')
    It is between  tx[sa[1]:] == 'ana' < 'anana' == tx[sa[2]:]
    """
    tx = text
    size = len(tx)
    step = min(max(_step, 1), len(tx))
    sa = list(range(len(tx)))
    sa.sort(key=lambda i: tx[i:i + step])
    grpstart = size * [False] + [True]  # a boolean map for iteration speedup.
    # It helps to skip yet resolved values. The last value True is a sentinel.
    rsa = size * [None]
    stgrp, igrp = '', 0
    for i, pos in enumerate(sa):
        st = tx[pos:pos + step]
        if st != stgrp:
            grpstart[igrp] = (igrp < i - 1)
            stgrp = st
            igrp = i
        rsa[pos] = igrp
        sa[i] = pos
    grpstart[igrp] = (igrp < size - 1 or size == 0)
    while grpstart.index(True) < size:
        # assert step <= size
        nextgr = grpstart.index(True)
        while nextgr < size:
            igrp = nextgr
            nextgr = grpstart.index(True, igrp + 1)
            glist = []
            for ig in range(igrp, nextgr):
                pos = sa[ig]
                if rsa[pos] != igrp:
                    break
                newgr = rsa[pos + step] if pos + step < size else -1
                glist.append((newgr, pos))
            glist.sort()
            for ig, g in groupby(glist, key=itemgetter(0)):
                g = [x[1] for x in g]
                sa[igrp:igrp + len(g)] = g
                grpstart[igrp] = (len(g) > 1)
                for pos in g:
                    rsa[pos] = igrp
                igrp += len(g)
        step *= 2
    del grpstart
    del rsa
    return sa

def SA_search(P, W, sa):
    lp = len(P)
    n = len(sa)
    l = 0; r = n
#     print(l,r)
    while l < r:
        mid = int((l+r) / 2)
        a = sa[mid]
        if P > W[a : a + lp]:
            l = mid + 1
        else:
            r = mid
    s = l; r = n
    while l < r:
        mid = int((l+r) / 2)
        a = sa[mid]
        if P < W[a : a + lp]:
            r = mid
        else:
            l = mid + 1
    return [sa[i] for i in range(s, r)]

def find_homologous_peptide(P):
    """
    Gets a peptide and returns whether it has homolegous in the genome.
    If so, that peptide is discarded.
    """
    if len(SA_search('K' + P, W_aa_ambiguous, sa_ambiguous)) > 0:
        return False
    if len(SA_search('R' + P, W_aa_ambiguous, sa_ambiguous)) > 0:
        return False
    if len(SA_search('*' + P, W_aa_ambiguous, sa_ambiguous)) > 0:
        return False
    return True

def create_modified_seq(modified_seq, destination):
    """
    Input: base sequence with probabilities of substitution for each AA, observed destination AA.
    Output: modified sequence.
    (ACY(0.1)KFL(0.9)S, Y) -> ACYKFYS
    """
    possible_sites = re.findall('\(([^\)]+)\)', modified_seq)
    best_site = np.argmax([float(i) for i in possible_sites])
    modified_pos_prime = [m.start()-1 for m in re.finditer('\(',modified_seq) ][best_site]
    modified_pos = len(re.sub('\(([^\)]+)\)', '', modified_seq[:modified_pos_prime]))
    base_seq = re.sub('\(([^\)]+)\)', '', modified_seq)
    return base_seq[:modified_pos] + destination + base_seq[modified_pos+1:]

In [4]:
bases = 'TCAG'
codons = [a+b+c for a in bases for b in bases for c in bases]
amino_acids = 'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG' #corresponds to codons
RC = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}

codon_table = get_codon_table()
inverted_codon_table = get_inverted_codon_table()
inverted_codon_table['L'] = inverted_codon_table['L'] + inverted_codon_table['I']
MW_dict = {"G": 57.02147, 
            "A" : 71.03712, 
            "S" : 87.03203, 
            "P" : 97.05277, 
            "V" : 99.06842, 
            "T" : 101.04768, 
            "I" : 113.08407, 
            "L" : 113.08407, 
            "N" : 114.04293, 
            "D" : 115.02695, 
            "Q" : 128.05858, 
            "K" : 128.09497, 
            "E" : 129.0426, 
            "M" : 131.04049, 
            "H" : 137.05891,
            "F" : 147.06842, 
            "R" : 156.10112, 
            "C" : 160.030654, #CamCys
            "Y" : 163.0633,
            "W" : 186.07932,
            }# Molecular mass of amino acid residues

subs_dict = { i+' to '+j : MW_dict[j] - MW_dict[i] for i in MW_dict for j in MW_dict if i!=j}
del subs_dict['L to I']
del subs_dict['I to L']
# Leucine and isoleucine residues have the same molecular weight, and changes in both amino acids cannot be detected.

for k,v in list(subs_dict.items()): # unifies I and L
    if k[-1]=='I':
        subs_dict[k+'/L'] = v
        del subs_dict[k]
        del subs_dict[k[:-1]+'L']

sorted_subs, sorted_sub_masses = zip(*sorted(subs_dict.items(), key= lambda x: x[1]))

In [5]:
""" Loads CDS fasta file and builds records of genes within it """
record_list = []
translated_record_list = []
names_list = []
record_dict = {}
boundaries_aa = [0]
W_codons = []
for record in SeqIO.parse(open(path_to_fasta,'rU'),'fasta'):
    record.seq = record.seq.upper()    
    if is_gene(record):
        translation = str(record.seq.translate())
        bits = record.description.split(' ')
        for i in bits:
            if 'gene=' in i:
                record.name = i.split('=')[-1].strip(']')
        record_list.append(record)
        translated_record_list.append(translation) # amino acid sequence
        names_list.append(record.name) # gene name
        record_dict[record.name] = record
        boundaries_aa.append(boundaries_aa[-1]+len(translation))
        W_codons.extend(list(codonify(record.seq)))

In [6]:
boundaries_aa = np.array(boundaries_aa[1:]) # an array annotating the genes' cumulative length
W_aa = ''.join(translated_record_list)
sa = suffix_array(W_aa)
W_aa_ambiguous = W_aa.replace('I','L')
sa_ambiguous = suffix_array(W_aa_ambiguous)

AAs = 'ACDEFGHIKLMNPQRSTVWY'
sites = list(AAs)+['nterm','cterm']

In [7]:
dp_columns = ['Raw file', 'Charge', 'm/z', 'Retention time', 'DP base sequence',
              'DP mass difference', 'DP time difference', 'DP PEP', 'DP probabilities', 
              'DP positional probability', 'DP decoy'] # Column names may differ depending on the MaxQuant version used

dp_df = pd.read_csv(path_to_allPeptides,sep="\t",usecols=dp_columns)
dp = dp_df[pd.notnull(dp_df['DP mass difference'])]
dp.reset_index(drop=True, inplace=True)

dp['DPMD'] = dp['DP mass difference']
dp['DPAA_noterm'] = dp['DP probabilities'].map(refine_localization_probabilities)
dp['nterm'] = dp['DP probabilities'].map(n_term_probability) # p(N-term AA was substituted)
dp['cterm'] = dp['DP probabilities'].map(c_term_probability)
dp['prot_nterm'] = dp['DP base sequence'].map(is_prot_nterm) # Does the peptide come from the N-term of the protein
dp['prot_cterm'] = dp['DP base sequence'].map(is_prot_cterm)

In [1]:
""" Handles mass differences that can be explained as PTMs """
danger_mods = pd.read_pickle(path_to_danger_mods)

dp['danger'] = False
for mod in danger_mods.iterrows():
    mod = mod[1]
    position = mod['position']
    site = mod['site']
    delta_m = mod['delta_m']
    
    mass_filter = (delta_m - (2*tol) < dp.DPMD) & (dp.DPMD < delta_m + (2*tol))
    
    term_filter = True
    if position == 'Protein N-term':
        term_filter = (dp.nterm > n_term_prob_cutoff) & dp.prot_nterm
    elif position == 'Protein C-term':
        term_filter = (dp.cterm > c_term_prob_cutoff) & dp.prot_cterm
    elif position == 'Any N-term':
        term_filter = dp.nterm > n_term_prob_cutoff
    elif position == 'Any C-term':
        term_filter = dp.cterm > c_term_prob_cutoff
    
    site_filter = True
    if site in amino_acids:
        site_filter = dp.DPAA_noterm.str.contains(site)
    
    dp.loc[site_filter & term_filter & mass_filter, 'danger'] = True

In [ ]:
""" mass differences that could be explained as substitution """
dp['substitution'] = False
for i in sorted(subs_dict.keys()):
    delta_m = subs_dict[i]
    original_aa = i[0]
    dp.loc[(dp.DPMD > delta_m - tol) & (dp.DPMD < delta_m + tol) & (dp['DPAA_noterm'] == original_aa) & (dp['DP positional probability']>0.95) & ~dp['danger'], 'substitution'] = i

In [ ]:
tmp_df = dp.copy()
# remove PTM
print('Original info')
tmp_df = tmp_df[tmp_df['danger']!=True]
print('remove PTM')

print('Could be explained by subtitutions')

for i in sorted(subs_dict.keys()):
    delta_m = subs_dict[i]
    original_aa = i[0]
    tmp_df.loc[(tmp_df.DPMD > delta_m - tol) & (tmp_df.DPMD < delta_m + tol) & (tmp_df['DPAA_noterm'] == original_aa), 'substitution'] = True
    
tmp_df = tmp_df[tmp_df['substitution']==True]

In [ ]:
"""
Create mask for mispairing. A binary dataframe indicating for each codon the AAs encoded by near-cognate codons. 
"""            
mask = pd.DataFrame(data = False, index = codons, columns=list('ACDEFGHKLMNPQRSTVWY'),dtype=float)    
for label in codons:
    near_cognates = np.array([hamming(i,label)==1 for i in codons]) # 相近密码子
    reachable_aa = set(np.array(list(amino_acids))[near_cognates])
    mask.loc[label] =[i in reachable_aa for i in 'ACDEFGHKLMNPQRSTVWY']

for label in mask.index: # removes "near-cognates" that encodes the same AA
    for col in mask.columns:
        if label in inverted_codon_table[col]:
            mask.loc[label, col] = float('NaN')

In [ ]:
subs = dp[dp['substitution']!=False].copy() # after filtering
subs['proteins'] = subs['DP base sequence'].map(find_proteins) 
subs['protein'] = subs['proteins'].map(lambda x: x.split(' ')[0] if len(x)>0 else float('NaN'))
subs = subs[pd.notnull(subs['protein'])] #mismatching files

subs['codons'] = float('NaN')
subs.loc[ subs['DP positional probability'] > positional_probability_cutoff, 'codons' ] = subs[ subs['DP positional probability'] > positional_probability_cutoff ]['DP probabilities'].map(fetch_best_codons)
subs['codon'] = subs['codons'].map(lambda x: x.split(' ')[0] if len(set(x.split(' ')))==1 else float('NaN')) 
subs['destination'] = subs['substitution'].map(lambda x: x[-1] if x else False)
subs['origin'] = subs['substitution'].map(lambda x: x[0] if x else False)
subs['mispairing'] = subs.apply(is_mispairing,1)    

subs['positions'] = subs.apply(lambda row : 
        find_positions_local(row['DP probabilities'],row['proteins']),
        axis=1)
subs['position'] = subs['positions'].map(lambda x: int(x.split(' ')[0]) if len(set(x.split(' ')))==1 else float('NaN')) 
subs['modified_sequence'] = subs.apply(lambda row : create_modified_seq(row['DP probabilities'], row['destination']), axis=1)
subs['modified_sequence'] = subs['modified_sequence'].map(lambda x: x.replace('I','L'))
subs = subs[subs['modified_sequence'].map(lambda x: find_homologous_peptide(x))] 

subs.sort_values('DP PEP', inplace = True)
subs['decoy'] = pd.notnull(subs['DP decoy'])
cut_off = np.max(np.where(np.array([i/float(j) for i,j in zip(subs['decoy'].cumsum(), range(1,len(subs)+1))])<fdr))
subs = subs.iloc[:cut_off+1]

subs_raw = subs.copy()

In [ ]:
subs.head(5)

In [ ]:
subs_reps = subs.copy()
subs_reps = subs_reps[subs_reps['DP decoy']!='+']
subs_reps = subs_reps[pd.notnull(subs_reps['codon'])]

# 将样本和文件进行对应
columns_summary = ['Raw file', 'Experiment']
summary = pd.read_csv(path_to_summary,sep="\t",usecols = columns_summary)
summary.columns = ['Raw file','Sample']
subs_reps = pd.merge(subs_reps,summary,on='Raw file')

usedCategories = [
    '0_2h','4_6h','10_12h','18_20h','44_46h','66_68h','83_85h','l3',
    'p1', 'p2','p3', 'p4', 'p5','vf_male','vf_female','male','female'
]
len(usedCategories)

In [ ]:
subs_reps['condition'] = subs_reps['Sample'].apply(lambda x: '_'.join(x.split('_')[:-1]))
subs_reps = subs_reps[subs_reps['condition'].isin(usedCategories)]

In [ ]:
editing_related_sites_df = pd.read_csv("editing_related_sites_AllSamples.csv",sep="\t")
subs_reps = subs_reps[~(subs_reps['proteins'].isin(editing_related_sites_df['proteins']))&
               ~(subs_reps['position'].isin(editing_related_sites_df['position']))]

subs_reps_tmp = subs_reps[['Sample','substitution','protein','codon','position']].drop_duplicates().copy()

subs_reps_tmp_count = subs_reps_tmp.groupby(['substitution','protein','codon','position'])['Sample'].count().reset_index()

subs_reps_tmp_count = subs_reps_tmp_count[subs_reps_tmp_count['substitution'] != 'A to I/L'] # this substitution is beyond detection

sample_count = subs_reps_tmp_count['Sample'].value_counts().reset_index()
sample_count.columns = ['DetectedSamples','Count']

tmp_df = pd.DataFrame(columns=['DetectedSamples'])
tmp_df['DetectedSamples'] = [i for i in range(1,69)]
sample_count = pd.merge(sample_count,tmp_df,how='outer')
sample_count.fillna(0,inplace=True)
sample_count['log2Count'] = np.log2(sample_count['Count']+1)

In [ ]:
sample_count_new = sample_count[sample_count['DetectedSamples']<12][['DetectedSamples','Count']]
new_row = {'DetectedSamples':'12~68','Count':sample_count[sample_count['DetectedSamples']>11]['Count'].sum()}
sample_count_new = sample_count_new.append(new_row,ignore_index=True)
sample_count_new['DetectedSamples'] = sample_count_new['DetectedSamples'].astype(str)
sample_count_new.head(3)

In [ ]:
plt.figure(figsize=(6,3))
sns.barplot(x='DetectedSamples',y='Count',data=sample_count_new,color='steelblue')
plt.xlabel("the number of detected samples")
plt.ylabel("the count of substitutions")
plt.tight_layout()

In [ ]:
subs_reps_tmp_clean = subs_reps_tmp_count[subs_reps_tmp_count['Sample']<4] 

del subs_reps_tmp_clean['Sample']

# print(subs_reps.shape)

subs_reps_clean = pd.merge(subs_reps,subs_reps_tmp_clean,on=['substitution','protein','codon','position'])

# print(subs_reps_clean.shape)

In [ ]:
# Error Count
subs_reps = subs_reps_clean.copy()
subs_reps['Sample'] = subs_reps['Raw file'].apply(lambda x:re.search(r"set[12]_(.*)",x).group(1))

count_info = subs_reps['Sample'].value_counts().reset_index()
count_info.columns = ['Sample','Count']
count_info['label'] = count_info['Sample'].apply(lambda x: '_'.join(x.split('_')[:-1]))


LabelList = ['0_2h','4_6h','10_12h','18_20h',
             'L1_44_46h','L2_66_68h','eL3_83_85h','lateL3',
             'P1','P2','P3','P4','P5',
             'vF_male','vF_female','F_male','F_female']

count_info['label'] = count_info['label'].astype("category").cat.set_categories(LabelList)
count_info.sort_values("label",inplace=True)

plt.figure(figsize=(6,3))
sns.barplot(x='label',y='Count',data=count_info,color='steelblue',alpha=0.8,ci=68)
plt.xlabel("")
plt.ylabel("Substitution sites detected")
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
def prepare_count_matrix(df):

    matrix = pd.DataFrame(data = 0, index = codons, columns=list('ACDEFGHKLMNPQRSTVWY'),dtype=float)
    df = df[df['DP decoy']!='+']
    df = df[pd.notnull(df['codon'])] # filter 
    df['codon'] = df['codon'].map(lambda x: x.replace('T','U'))
    for label in matrix.index:
        if codon_table[label] == '*':
            matrix.loc[label] = float('NaN')
        for col in matrix.columns:
            if (label in inverted_codon_table[col]) or (codon_table[label] +' to '+col in exact_PTM_spec_list):
                matrix.loc[label, col] = float('NaN')
    subs_agg = pd.DataFrame(df.groupby(['protein','position','origin','destination','codon']).groups.keys(), columns=['protein','position','origin','destination','codon'])

    for x, l in subs_agg.groupby(['codon', 'destination']).groups.items():
        codon, destination = x
        if (codon in matrix.index) and pd.notnull(matrix.loc[codon,destination]):
            matrix.loc[codon,destination] = len(l)
    matrix.rename(columns={"L": "I/L"},inplace=True)
    return matrix
    
def probe_mismatch(codon1, codon2, pos, spec):
    origin, destination = spec
    for i in range(3):
        if i == pos:
            if codon1[i] != origin or codon2[i] != destination:
                return False
        else:
            if codon1[i] != codon2[i]:
                return False
    return True

bases = 'UCAG'
codons = [a+b+c for a in bases for b in bases for c in bases]

amino_acids = 'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'
RC = {'A':'U', 'C':'G', 'G':'C', 'U':'A'}

codon_table = get_codon_table()
inverted_codon_table = get_inverted_codon_table()
inverted_codon_table['L'] = inverted_codon_table['L'] + inverted_codon_table['I']
tol = 0.005

aas_sorted_by_mass = [i[0] for i in sorted(MW_dict.items(),key=lambda x:x[1])]
danger_mods = pd.read_pickle('danger_mods')
exact_PTM_spec = pd.DataFrame(index = aas_sorted_by_mass,
                              columns = aas_sorted_by_mass,
                              dtype = int)

for aa1 in MW_dict.keys():
    for aa2 in MW_dict.keys():
        delta_m = MW_dict[aa2] - MW_dict[aa1]
        exact_PTM_spec.loc[aa1,aa2]=len(danger_mods[(danger_mods['delta_m']<delta_m + 0.0005) & (danger_mods['delta_m']>delta_m - 0.0005) & (danger_mods['site']==aa1)]) > 0

exact_PTM_spec_list = [str(i) + ' to ' + str(j) for i in aas_sorted_by_mass for j in  aas_sorted_by_mass if exact_PTM_spec.loc[i,j]] 

mask = pd.DataFrame(data = False,
                    index = codons,
                    columns = list('ACDEFGHKLMNPQRSTVWY'),
                    dtype = float)

for label in codons:
    near_cognates = np.array([hamming(i,label)==1 for i in codons])
    reachable_aa = set(np.array(list(amino_acids))[near_cognates])
    mask.loc[label] =[i in reachable_aa for i in 'ACDEFGHKLMNPQRSTVWY']
    
for label in mask.index:
    if codon_table[label] == '*':
        mask.loc[label]=float('NaN')
    for col in mask.columns:
        if (label in inverted_codon_table[col]) or (codon_table[label] +' to '+col in exact_PTM_spec_list):
            mask.loc[label, col] = float('NaN')

subs_reps = subs_reps[~subs_reps.decoy]

data = prepare_count_matrix(subs_reps)
data.to_csv("../Results/data.csv",sep="\t")

In [ ]:
data.sum().sum()

In [ ]:
log_data = np.log2(data+1)
m = log_data.max().max()

log_data_tmp = log_data.copy()
log_data_tmp.reset_index(inplace=True)
log_data_tmp['protein'] = log_data_tmp['index'].apply(lambda x: str(Seq.Seq(x).translate()))
log_data_tmp['label'] = log_data_tmp.apply(lambda x: x['protein']+'-'+x['index'],axis=1)
log_data_tmp.index = log_data_tmp['label']
log_data_tmp.drop(columns=['index','protein','label'],inplace=True)
log_data_tmp.head(2)

sns.set_style('darkgrid')

font = {'size': 13}
# sns.set_style({'font.family':'sans-serif', 'font.serif':['Arial']})
fig, ax = plt.subplots(figsize= (13,13))
ax = sns.heatmap(log_data_tmp,square = True, cmap = 'Reds', ax = ax, cbar_kws = {"shrink": .2},linewidths=2)
ax.set_ylabel('original codon', fontdict=font)
ax.set_xlabel('destination amino acid', fontdict=font)
ax.tick_params(labelsize=12)

### Error Rate Calculate

In [ ]:
mf = pd.read_csv(path_to_matched_features,
                 sep = '\t')
mf.replace(0, np.nan, inplace = True)

pep_head = pd.read_csv(path_to_peptides,
                  sep = '\t', 
                  nrows = 10,
                  index_col = 'Sequence')

intensities = [i for i in pep_head.columns if 'Intensity ' in i]
samples = [i.split()[-1] for i in intensities]

pep_columns = ['Sequence', 'Intensity', 'Evidence IDs'] + intensities

pep = pd.read_csv(path_to_peptides,
                  sep = '\t', 
                  usecols = pep_columns,
                  index_col = 'Sequence')

pep.replace(0, np.nan, inplace = True)

In [ ]:
subs_reps = subs_raw.copy()
subs_reps = subs_reps[subs_reps['DP decoy']!='+']
subs_reps = subs_reps[pd.notnull(subs_reps['codon'])]

columns_summary = ['Raw file', 'Experiment']
summary = pd.read_csv(path_to_summary,sep="\t",usecols = columns_summary)
summary.columns = ['Raw file','Sample']
subs_reps = pd.merge(subs_reps,summary,on='Raw file')

subs_reps['condition'] = subs_reps['Sample'].apply(lambda x: '_'.join(x.split('_')[:-1]))
# remove editing related sites
editing_related_sites_df = pd.read_csv("../Data/editing_related_sites_AllSamples.csv",sep="\t")
subs_reps = subs_reps[~(subs_reps['proteins'].isin(editing_related_sites_df['proteins']))&
               ~(subs_reps['position'].isin(editing_related_sites_df['position']))]

subs_reps_tmp = subs_reps[['Sample','substitution','protein','codon','position']].drop_duplicates().copy()

subs_reps_tmp_count = subs_reps_tmp.groupby(['substitution','protein','codon','position'])['Sample'].count().reset_index()

subs_reps_tmp_count = subs_reps_tmp_count[subs_reps_tmp_count['substitution'] != 'A to I/L'] # this substitution is beyond detection

subs_reps_tmp_clean = subs_reps_tmp_count[subs_reps_tmp_count['Sample']<6]


del subs_reps_tmp_clean['Sample']
subs_reps_clean = pd.merge(subs_reps,subs_reps_tmp_clean,on=['substitution','protein','codon','position'])

In [ ]:
def find_cds_seq(base_seq,protein):
    """
    """
#     print(base_seq,protein)
    s = record_dict[protein].seq
    tmp_pep = s.translate()
    seq_i = s.translate().find(base_seq)
    tmp_cds = str(s[seq_i*3:(len(base_seq)+seq_i)*3])
    return(tmp_cds)

def find_CDSs_local(base_seq, proteins):
    """
    """
    CDSs = []
    if proteins == '': # if peptide could not mapped into any known protein
        return False
    else:
        
        for prot in proteins.split(" "):
            CDSs.append(str(find_cds_seq(base_seq, prot)))
        return " ".join(CDSs)

In [ ]:
pep_df = pep.copy()
pep_df.reset_index(inplace=True)
pep_df['CDSs'] = pep_df['Sequence'].apply(lambda x: find_CDSs_local(x,find_proteins(x))) # may take 15 mins?

pep_df = pep_df[pep_df['CDSs'] != False]
pep_df['CDS'] = pep_df['CDSs'].apply(lambda x: x.split(' ')[0] if len(set(x.split(' '))) == 1 else False)
pep_df = pep_df[pep_df['CDS']!=False]

In [ ]:
Codons = [i.replace('U','T') for i in codons]
# codons.replace('U','T')

BP_Intensity = {}

for sample in intensities:
    BP_Intensity[sample] = {}
    for Codon in Codons:
        BP_Intensity[sample][Codon] = 0
        
# assign intensity into each codon of each experiment
for index,row in pep_df.iterrows():
    for codon in codonify(row['CDS']):
        for sample in intensities:
#             print(row[sample])
            if not np.isnan(row[sample]):
#                 print(row[sample])
                BP_Intensity[sample][codon] += row[sample]

In [ ]:
columns_summary = ['Raw file', 'Experiment']
summary = pd.read_csv(path_to_summary,sep="\t",usecols = columns_summary)
summary.columns = ['Raw file','Sample']

"""
matchedFeatured.txt contains features that weren't identified and appeared in more than one sample.
Those features have both uncalibrated and calibrated retention time.
subs, which is derived from allPeptides.txt, has only uncalibrated retention time;
to extract features from mf.txt, retention times are calibrated using evidence.txt
"""
columns_evidence = ['Raw file', 'Retention time', 'Calibrated retention time', 'Intensity', 'Charge']

evidence = pd.read_csv(path_to_evidence,
                       sep = '\t', usecols = columns_evidence)

# evidence['Sample'] = evidence['Raw file'].map(lambda x: x.split('_')[-2]) # 
# Sample 
evidence = pd.merge(evidence,summary,on='Raw file')

calibrate = {}
for i,j in evidence.groupby('Raw file'): # RT for each file (fraction) is being calibrated
    calibrate[i] = interp1d(j['Retention time'], j['Calibrated retention time'],fill_value="extrapolate")

In [ ]:
subs_tmp = subs_reps_clean.copy()

subs_tmp['modified_sequence'] = subs_tmp.apply(lambda row : create_modified_seq(row['DP probabilities'], row['destination']), axis=1) 
subs_tmp['base RT'] =  subs_tmp['Retention time'] - subs_tmp['DP time difference']

for i,j in subs_tmp.groupby('Raw file'):
    subs_tmp.loc[j.index, 'Calibrated retention time'] = subs_tmp.loc[j.index, 'Retention time'].map(lambda x: calibrate[i](x))
    subs_tmp.loc[j.index, 'Calibrated base RT'] = subs_tmp.loc[j.index, 'base RT'].map(lambda x: calibrate[i](x))

subs_tmp['Calibrated DPTD'] = subs_tmp['Calibrated retention time'] - subs_tmp['Calibrated base RT']
subs_tmp['Base m/z'] = subs_tmp['m/z'] - (subs_tmp['DPMD']/subs_tmp['Charge'])

In [ ]:
subs_codon = subs_tmp[~subs_tmp['codon'].isnull()] 

DP_Intensity = {}
DP_DetectCount = {} # 

for sample in intensities:
    DP_Intensity[sample] = {}
    DP_DetectCount[sample] = {}
    for Codon in Codons:
        DP_Intensity[sample][Codon] = 0
        DP_DetectCount[sample][Codon] = 0

gb_codon = subs_codon.groupby(['Charge','DP base sequence','modified_sequence'])

for i,j in gb_codon:
    charge, base_sequence, modified_sequence = i
    ref_dp = j.iloc[j['DP PEP'].argmin()] # 
    charge, dp_mz, base_sequence, dp_rt, codon = ref_dp[['Charge','m/z','DP base sequence','Calibrated retention time','codon']]
    
    f1 = mf['Calibrated retention time'] < dp_rt + rt_tol
    f2 = mf['Calibrated retention time'] > dp_rt - rt_tol
    f3 = mf['m/z'] < dp_mz * (1 + mz_tol)
    f4 = mf['m/z'] > dp_mz * (1 - mz_tol)
    f5 = mf['Charge'] == charge
    
    m = mf[f1 & f2 & f3 & f4 & f5] #
    l = len(m)
    
    if l>=1:
        for sample in intensities:
            if not np.isnan(m[sample].values[0]):
                DP_Intensity[sample][codon] += m[sample].values[0]
                DP_DetectCount[sample][codon] += 1

In [ ]:
ErrorRate_codon = {}

for sample in intensities:
    ErrorRate_codon[sample] = {}
    for Codon in Codons:
        ErrorRate_codon[sample][Codon] = DP_Intensity[sample][Codon]/(BP_Intensity[sample][Codon]+1+DP_Intensity[sample][Codon])
        
ErrorRate_codon_df = pd.DataFrame(ErrorRate_codon)

In [ ]:
# 17 Samples
columns_df = pd.DataFrame(ErrorRate_codon_df.columns)
columns_df.columns = ['label']
columns_df['condition'] = columns_df['label'].apply(lambda x: '_'.join(x.split(' ')[1].split('_')[:-1]))
usedCategories = [
    '0_2h','4_6h','10_12h','18_20h','44_46h','66_68h','83_85h','l3',
    'p1', 'p2','p3', 'p4', 'p5','vf_male','vf_female','male','female'
]
columns_df = columns_df[columns_df['condition'].isin(usedCategories)]

ErrorRate_codon_used = ErrorRate_codon_df.loc[:,list(columns_df['label'].values)]
ErrorRate_codon_used.to_csv("../Results/ErrorRate_tmp.csv")